In [2]:
import re 
import csv
import json
import nltk
import sklearn
import pandas as pd
from pprint import pprint
from bs4 import BeautifulSoup 


### Read data from json file and Merge them into Pandas Data Frame


In [26]:
with open('drama_pos_reviews_100.json') as json_data:
    posReviewsDrama = json.load(json_data)
    
with open('drama_neg_reviews_100.json') as json_data:
    negReviewsDrama = json.load(json_data)
    
with open('action_love_reviews_100.json') as json_data:
    posReviewsAction = json.load(json_data)
    
with open('action_hate_reviews_100.json') as json_data:
    negReviewsAction  = json.load(json_data) 

In [3]:
with open('action_love_reviews_100.json') as json_data:
    posReviewsAction = json.load(json_data)

In [5]:
len(posReviewsAction)

0

In [74]:
posReList = []
for reviewlist in posReviewsDrama:
    if reviewlist is not None:
        for review in reviewlist:
            posReList.append([review['author'],review['review'],review['categorie'],review['movie']])
            
posReviews = pd.DataFrame(posReList,columns=['id', 'reviews', 'sentiment','movie'])

In [79]:
negReList = []
for reviewlist in  negReviewsAction:
    if reviewlist is not None:
        for review in reviewlist:
            negReList.append([review['author'],review['review'],review['categorie'],review['movie']])
            
negReviews = pd.DataFrame(negReList,columns=['id', 'reviews', 'sentiment','movie'])

In [59]:
print(posReviews.shape)
print(negReviews.shape)

(20564, 4)
(20680, 4)


### Text Cleaning and Text Prepocessing

+ Convert the words to lower case
+ Remove punctuation / Remove Numbers (Any non-letters one)
  NB: emoji could be uesd to analysis sentiment as well in the case
+ Remove stopwords
+ Steaming words 
+ Split the words

In [42]:
posReviews['reviews'] = posReviews['reviews'].str.lower()

In [64]:
print(posReviews.loc[posReviews['id']=='ur1046460',])

Empty DataFrame
Columns: [id, reviews, sentiment, movie]
Index: []


In [82]:
print(negReviews.loc[negReviews['id']=='ur47295305',])

             id                                            reviews sentiment  \
381  ur47295305  If you love fast paced action, where the hero ...  negative   
382  ur47295305  If you love fast paced action, where the hero ...  negative   

         movie  
381  tt2911666  
382  tt2911666  


In [80]:
negReviews[negReviews.duplicated()]

,id,reviews,sentiment,movie
300,ur62513246,This movie clearly wants to be in the original...,negative,tt2911666
382,ur47295305,"If you love fast paced action, where the hero ...",negative,tt2911666
703,ur45239062,I don't watch a lot of TV but I went to a frie...,negative,tt3107288
998,ur28788765,After seeing the 8.1 score and reading some re...,negative,tt2193021
2491,ur26240657,SUICIDE SQUAD - My plan for today was to watch...,negative,tt1386697
3078,ur1406466,"And lo, did Disney acquire the rights to argua...",negative,tt3748528
